In [108]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import linregress
from yelp.client import Client
import json
from ratelimit import limits
import requests
import csv
import random
from keys import my_api_key


df = pd.read_csv("../sub-est2019_all.csv", engine='python')
rural_city = df[(df["POPESTIMATE2019"]< 50000) & (df["POPESTIMATE2019"] > 1000)]
rural_df = rural_city[["NAME", "STNAME", "POPESTIMATE2019"]]
urban_city = df[df["POPESTIMATE2019"]>= 50000]
urban_df = urban_city[["NAME", "STNAME", "POPESTIMATE2019"]]
urban_index = urban_df.loc[urban_df["NAME"] == urban_df["STNAME"]].index
urban_clean = urban_df.drop(urban_index)
random_urban_city = urban_clean[["NAME", "STNAME","POPESTIMATE2019"]].sample(n=100, random_state=1)
random_rural_city = rural_df[["NAME", "STNAME", "POPESTIMATE2019"]].sample(n=100, random_state=1)


In [163]:
headers = {'Authorization': 'Bearer %s' % my_api_key}

rural_dict = {
    "Searched": [],
    "City": [],
    "State": [],
    "Lat": [],
    "Lng": [],
    "Restaurant": [],
    "Category": [],
    "Category 2": [],
    "Category 3": [],
    "Transactions":[]
}

url='https://api.yelp.com/v3/businesses/search'
 
for city, state in zip(random_rural_city["NAME"], random_rural_city["STNAME"]):
    params = {
        'limit': 50,
        'offset': 50,
        'location': f"{city}, {state}",
        'radius': 4000
             }
    response = requests.get(url, params = params, headers = headers).json()
    for business in range(len(response["businesses"])):
        try:
            rural_dict["Searched"].append(f"{city}, {state}")
            rural_dict["City"].append(response["businesses"][business]["location"]["city"])
            rural_dict["State"].append(response["businesses"][business]["location"]["state"])
            rural_dict["Lat"].append(response["businesses"][business]["coordinates"]["latitude"])
            rural_dict["Lng"].append(response["businesses"][business]["coordinates"]["longitude"])
            rural_dict["Restaurant"].append(response["businesses"][business]["name"])
#             rural_dict["Category"].append([response["businesses"][business]["categories"][i]["title"] for i in range(len(response["businesses"][business]["categories"]))])
            rural_dict["Category"].append(response["businesses"][business]["categories"][0]["title"])
            rural_dict["Category 2"].append(response["businesses"][business]["categories"][1]["title"])
            rural_dict["Category 3"].append(response["businesses"][business]["categories"][2]["title"])
            rural_dict["Transactions"].append(response["businesses"][business]["transactions"])
        except KeyError:
            rural_dict["Searched"].append("No City Found")
            rural_dict["City"].append("No City Found")
            rural_dict["State"].append("No State Found")
            rural_dict["Lat"].append("No Lat Found")
            rural_dict["Lng"].append("No Lng Found")
            rural_dict["Restaurant"].append("No Restaurant Found")
            rural_dict["Category"].append("No Categories")
            rural_dict["Transactions"].append("No Transactions Found")
        except IndexError:
            rural_dict["Category 2"].append("")
            rural_dict["Category 3"].append("")

In [164]:
rural = pd.DataFrame.from_dict(rural_dict, orient='index')
rural = rural.transpose()
rural.head(30)

,Searched,City,State,Lat,Lng,Restaurant,Category,Category 2,Category 3,Transactions
0,"Salem city, Virginia",Salem,VA,37.2905,-80.0764,McDonald's,Fast Food,Burgers,Coffee & Tea,[delivery]
1,"Salem city, Virginia",Salem,VA,37.2882,-80.0871,KFC,Fast Food,Chicken Wings,Chicken Shop,"[delivery, pickup]"
2,"Salem city, Virginia",Salem,VA,37.2869,-80.0952,Arby's,Fast Food,Sandwiches,,[delivery]
3,"Salem city, Virginia",Salem,VA,37.2888,-80.0858,Taco Bell,Fast Food,,,[]
4,"Salem city, Virginia",Salem,VA,37.2884,-80.092,Waffle House,Breakfast & Brunch,Mexican,American (New),[]
5,"Salem city, Virginia",Salem,VA,37.2868,-80.0624,Papa John's Pizza,Pizza,,,[]
6,"Salem city, Virginia",Salem,VA,37.2711,-80.037,Long John Silver's,Seafood,Diners,,[]
7,"Salem city, Virginia",Salem,VA,37.289,-80.0875,Dunkin',Donuts,,,[]
8,"Salem city, Virginia",Salem,VA,37.2873,-80.0944,Hardee's,Fast Food,Fast Food,,[]
9,"Salem city, Virginia",Salem,VA,37.2872,-80.0498,Pizza Den,Pizza,,Italian,[]


In [116]:
headers = {'Authorization': 'Bearer %s' % my_api_key}

urban_dict = {
    "Searched": [],
    "City": [],
    "State": [],
    "Lat": [],
    "Lng": [],
    "Restaurant": [],
    "Category": [],
    "Category 2": [],
    "Category 3": [],
    "Transactions":[]
}

url='https://api.yelp.com/v3/businesses/search'
 
for city, state in zip(random_urban_city["NAME"], random_rural_city["STNAME"]):
    params = {
        'limit': 50,
        'offset': 50,
        'location': f"{city}, {state}",
        'radius': 4000
             }
    response = requests.get(url, params = params, headers = headers).json()
    for business in range(len(response["businesses"])):
        try:
            rural_dict["Searched"].append(f"{city}, {state}")
            rural_dict["City"].append(response["businesses"][business]["location"]["city"])
            rural_dict["State"].append(response["businesses"][business]["location"]["state"])
            rural_dict["Lat"].append(response["businesses"][business]["coordinates"]["latitude"])
            rural_dict["Lng"].append(response["businesses"][business]["coordinates"]["longitude"])
            rural_dict["Restaurant"].append(response["businesses"][business]["name"])
#             rural_dict["Category"].append([response["businesses"][business]["categories"][i]["title"] for i in range(len(response["businesses"][business]["categories"]))])
            rural_dict["Category"].append(response["businesses"][business]["categories"][0]["title"])
            rural_dict["Category 2"].append(response["businesses"][business]["categories"][1]["title"])
            rural_dict["Category 3"].append(response["businesses"][business]["categories"][2]["title"])
            rural_dict["Transactions"].append(response["businesses"][business]["transactions"])
        except KeyError:
            rural_dict["Searched"].append("No City Found")
            rural_dict["City"].append("No City Found")
            rural_dict["State"].append("No State Found")
            rural_dict["Lat"].append("No Lat Found")
            rural_dict["Lng"].append("No Lng Found")
            rural_dict["Restaurant"].append("No Restaurant Found")
            rural_dict["Category"].append("No Categories")
            rural_dict["Transactions"].append("No Transactions Found")
        except IndexError:
            rural_dict["Category 2"].append("")
            rural_dict["Category 3"].append("")

KeyError: 'businesses'

In [113]:
urban = pd.DataFrame.from_dict(urban_dict, orient='index')
urban = urban.transpose()
urban.head()

,Searched,City,State,Lat,Lng,Restaurant,Category,Transactions


In [152]:
urban.to_csv("../urban.csv")
rural.to_csv("../rural.csv")

In [ ]:
new_rural=rural["Category"].str.split(" ", n = 2, expand = True)
try:
    rural["Category 1"]= new_rural[0]
    rural["Category 2"]= new_rural[1]
    rural["Category 3"]= new_rural[2]
except KeyError:
    rural["Category 2"] = ""
    rural["Category 3"] = ""
rural.drop(columns =["Category"], inplace = True)


In [159]:
rural.head()

,Searched,City,State,Lat,Lng,Restaurant,Transactions,Category 1,Category 2,Category 3
0,"Salem city, Virginia",Salem,VA,37.2905,-80.0764,McDonald's,[delivery],NaN,,
1,"Salem city, Virginia",Salem,VA,37.2882,-80.0871,KFC,"[delivery, pickup]",NaN,,
2,"Salem city, Virginia",Salem,VA,37.2869,-80.0952,Arby's,"[delivery, pickup]",NaN,,
3,"Salem city, Virginia",Salem,VA,37.2888,-80.0858,Taco Bell,[delivery],NaN,,
4,"Salem city, Virginia",Salem,VA,37.2884,-80.092,Waffle House,[delivery],NaN,,
